In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

/root/miniconda3/envs/mdllm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/mdllm/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/miniconda3/envs/mdllm/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
# 将JSON文件转换为CSV文件
df = pd.read_json('./huanhuan.json')
ds = Dataset.from_pandas(df)

In [4]:
tokenizer = AutoTokenizer.from_pretrained('./qwen/Qwen-7B-Chat/', use_fast=False, trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eod_id
tokenizer

QWenTokenizer(name_or_path='./qwen/Qwen-7B-Chat/', vocab_size=151851, model_max_length=8192, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	
}

In [5]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["<|im_start|>system", "现在你要扮演皇帝身边的女人--甄嬛.<|im_end|>" + "\n<|im_start|>user\n" + example["instruction"] + example["input"] + "<|im_end|>\n"]).strip(), add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer("<|im_start|>assistant\n" + example["output"] + "<|im_end|>\n", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  # Qwen的特殊构造就是这样的
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map: 100%|██████████| 3729/3729 [00:01<00:00, 3246.71 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3729
})

In [7]:
import torch

model = AutoModelForCausalLM.from_pretrained('./qwen/Qwen-7B-Chat/', trust_remote_code=True, torch_dtype=torch.half, load_in_8bit=True, device_map="sequential")
model

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Loading checkpoint shards: 100%|██████████| 8/8 [00:05<00:00,  1.52it/s]


QWenLMHeadModel(
  (transformer): QWenModel(
    (wte): Embedding(151936, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (rotary_emb): RotaryEmbedding()
    (h): ModuleList(
      (0-31): 32 x QWenBlock(
        (ln_1): RMSNorm()
        (attn): QWenAttention(
          (c_attn): Linear8bitLt(in_features=4096, out_features=12288, bias=True)
          (c_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (core_attention_flash): FlashSelfAttention()
          (attn_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): RMSNorm()
        (mlp): QWenMLP(
          (w1): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (w2): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (c_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
        )
      )
    )
    (ln_f): RMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=151936, bias=False)
)

In [8]:
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype)

transformer.wte.weight torch.Size([151936, 4096]) torch.float16
transformer.h.0.ln_1.weight torch.Size([4096]) torch.float16
transformer.h.0.attn.c_attn.weight torch.Size([12288, 4096]) torch.int8
transformer.h.0.attn.c_attn.bias torch.Size([12288]) torch.float16
transformer.h.0.attn.c_proj.weight torch.Size([4096, 4096]) torch.int8
transformer.h.0.ln_2.weight torch.Size([4096]) torch.float16
transformer.h.0.mlp.w1.weight torch.Size([11008, 4096]) torch.int8
transformer.h.0.mlp.w2.weight torch.Size([11008, 4096]) torch.int8
transformer.h.0.mlp.c_proj.weight torch.Size([4096, 11008]) torch.int8
transformer.h.1.ln_1.weight torch.Size([4096]) torch.float16
transformer.h.1.attn.c_attn.weight torch.Size([12288, 4096]) torch.int8
transformer.h.1.attn.c_attn.bias torch.Size([12288]) torch.float16
transformer.h.1.attn.c_proj.weight torch.Size([4096, 4096]) torch.int8
transformer.h.1.ln_2.weight torch.Size([4096]) torch.float16
transformer.h.1.mlp.w1.weight torch.Size([11008, 4096]) torch.int8


In [17]:
for name, param in model.named_parameters():
    print(name)

base_model.model.transformer.wte.weight
base_model.model.transformer.h.0.ln_1.weight
base_model.model.transformer.h.0.attn.c_attn.base_layer.weight
base_model.model.transformer.h.0.attn.c_attn.base_layer.bias
base_model.model.transformer.h.0.attn.c_attn.lora_A.default.weight
base_model.model.transformer.h.0.attn.c_attn.lora_B.default.weight
base_model.model.transformer.h.0.attn.c_proj.base_layer.weight
base_model.model.transformer.h.0.attn.c_proj.lora_A.default.weight
base_model.model.transformer.h.0.attn.c_proj.lora_B.default.weight
base_model.model.transformer.h.0.ln_2.weight
base_model.model.transformer.h.0.mlp.w1.base_layer.weight
base_model.model.transformer.h.0.mlp.w1.lora_A.default.weight
base_model.model.transformer.h.0.mlp.w1.lora_B.default.weight
base_model.model.transformer.h.0.mlp.w2.base_layer.weight
base_model.model.transformer.h.0.mlp.w2.lora_A.default.weight
base_model.model.transformer.h.0.mlp.w2.lora_B.default.weight
base_model.model.transformer.h.0.mlp.c_proj.base_la

In [18]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["c_attn", "c_proj", "w1", "w2"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'c_attn', 'c_proj', 'w1', 'w2'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [19]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'c_attn', 'c_proj', 'w1', 'w2'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [20]:

model.print_trainable_parameters()

trainable params: 17,891,328 || all params: 7,739,215,872 || trainable%: 0.23117752878207867


In [21]:
args = TrainingArguments(
    output_dir="./output/Qwen",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    gradient_checkpointing=True,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True
)

In [22]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [23]:
trainer.train()

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


Step,Training Loss
10,3.343000
20,2.573700
30,2.459600
40,2.416400
50,2.458100
60,2.404100
70,2.505700
80,2.388600
90,2.384900
100,2.325200


Checkpoint destination directory ./output/Qwen/checkpoint-100 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/root/miniconda3/envs/mdllm/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory ./output/Qwen/checkpoint-200 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/root/miniconda3/envs/mdllm/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will 

TrainOutput(global_step=348, training_loss=2.2234837323769754, metrics={'train_runtime': 1542.3373, 'train_samples_per_second': 7.253, 'train_steps_per_second': 0.226, 'total_flos': 5.381300986918502e+16, 'train_loss': 2.2234837323769754, 'epoch': 2.98})

In [27]:
model.eval()
ipt = tokenizer("<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛.<|im_end|>\n<|im_start|>user\n{}<|im_end|>\n".format("你是谁？", "").strip() + "\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, 
                                max_length=512, 
                                do_sample=True, 
                                eos_token_id=tokenizer.eos_token_id, 
                                temperature=0.1)[0], 
                                skip_special_tokens=True)

Both `max_new_tokens` (=512) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


AssertionError: 

In [29]:
response, history = model.chat(tokenizer, 
                               "今天晚上吃什么?", 
                               history=[], 
                               system="现在你要扮演皇帝身边的女人--甄嬛.")
response

AssertionError: 